
En primer lugar vamos a preparar los datos (imagenes y descripciones) de una forma similar a la vista en clase. 

Lo intentaremos cambiando el dataset tespecto al que vimos en el ejemplo.

1.En primer lugar he intentado preparar un modelo con el dataset flickr 30k. Me he encontrado con las siguientes dificultades:

    -En este dataset no se incluían archivos con los títulos como el que utilizamos en el ejemplo de flickr8k. Para ello he adaptado dos nuevas funciones "load_titles" y "save_doc2" que permitieran crear una lista con los títulos de las fotos y grabarlas en un archivo de texto.

    -Finalmente, google colab no es capaz de escribir el archivo features.pkl para las 31.000 fotografías. He intentado jugar con el dataset parcialmente, añadiendo una instrucción de break en el loop de la VGG. Esta idea no ha resultado dado que al intentar asociar las caracteristicas a una foto mediante la funcion "load_photo_features" termina por dar error al llegar a una clave para la cual no tenemos sus características. 
    
    -Por estos motivos he tenido que abandonar el dataset flickr30k (después de un penoso intento con la CPU)
    

2.Una vez descartado flickr 30k y con idea de utilizar un dataset distinto al utilizado en clase, vamos a probar con el dataset Pascal Sentence.

[http://vision.cs.uiuc.edu/pascal-sentences/](http://vision.cs.uiuc.edu/pascal-sentences/)

    -Este dataset no tiene un enlace de descarga directa y por lo tanto hay que extraer las fotos y descripciones mediante web scrapping. Para ello utilizaremos el trabajo de un usuario de github.
    
    -Finalmente, después de asemejar todo al ejemplo de clase para utilizar el modelo los resultados no son satisfactorios, llegando a predicciones vacías "startseq -> endseq".
    
    -Al pelearme con las funciones del ejemplo de clase para que medio funcione todo al menos entiendo el proceso paso a paso mejor que al verlo en la sesión.
    
    -Dejo el codigo para que se vea el trabajo realizado ya que al menos he llegado a utilizar la LSTM y sacado un resultado. Sigo más abajo, ya con la flickr 8k como en clase.

In [3]:
!pip install pyquery

  Using cached https://files.pythonhosted.org/packages/09/c7/ce8c9c37ab8ff8337faad3335c088d60bed4a35a4bed33a64f0e64fbcf29/pyquery-1.4.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 5.9MB 4.7MB/s 


In [5]:
!pip install requests

In [18]:
!pip install urlparse

  Could not find a version that satisfies the requirement urlparse (from versions: )
No matching distribution found for urlparse


In [31]:
#!mkdir PascalSentenceDataSetOutput
#!mkdir PascalSentenceDataSetText
!ls -la

total 80
drwxr-xr-x 1 root root  4096 Jun 29 09:51 .
drwxr-xr-x 1 root root  4096 Jun 29 09:26 ..
drwx------ 4 root root  4096 Jun 29 09:33 .cache
drwxr-xr-x 3 root root  4096 Jun 29 09:33 .config
drwxr-xr-x 3 root root  4096 Jun 25 16:59 datalab
drwxr-xr-x 4 root root  4096 Jun 29 09:27 .forever
drwxr-xr-x 5 root root  4096 Jun 29 09:33 .ipython
drwx------ 3 root root  4096 Jun 29 09:27 .local
drwxr-xr-x 5 root root  4096 Jun 29 09:33 PascalSentenceDataset
drwxr-xr-x 2 root root 36864 Jun 29 09:48 PascalSentenceDataSetOutput
drwxr-xr-x 2 root root  4096 Jun 29 09:51 PascalSentenceDataSetText


In [0]:
#Codigo de github con algun cambio para que funcione como queremos y actualizado a py3

from urllib.parse import urljoin
from pyquery import PyQuery
import os
import requests
import csv

__author__ = 'rupy'

class PascalSentenceDataSet():

    DATASET_DIR = 'dataset/'
    SENTENCE_DIR = 'sentence/'
    PASCAL_SENTENCE_DATASET_URL = 'http://vision.cs.uiuc.edu/pascal-sentences/'

    def __init__(self):
        self.url = PascalSentenceDataSet.PASCAL_SENTENCE_DATASET_URL

    def download_images(self):
        dom = PyQuery(self.url)
        for img in dom('img').items():
            img_src = img.attr['src']
            category, img_file_name = os.path.split(img_src)

            # make category directories
            output_dir = "PascalSentenceDataSetOutput"
            #if not os.path.isdir(output_dir):
            #    os.mkdir(output_dir)

            # download image
            output = os.path.join(output_dir, img_file_name)
            print (output)
            if img_src.startswith('http'):
                img_url = img_src
            else:
                img_url = urljoin(self.url, img_src)
            if os.path.isfile(output):
                print ("Already downloaded, Skipping: %s" % output)
                continue
            print ("Downloading: %s" % output)
            with open(output,'wb') as f:

                while True:
                    result = requests.get(img_url)
                    raw = result.content
                    if result.status_code == 200:
                        f.write(raw)
                        break
                    print ("error occurred while fetching img")
                    print ("retry...")


    def download_sentences(self):
        dom = PyQuery(self.url)
        # tbody disappears in pyquery DOM
        for tr in dom('body>table>tr').items():
            img_src = tr('img').attr['src']
            category, img_file_name = os.path.split(img_src)

            # make category directories
            output_dir = "PascalSentenceDataSetText"
            #if not os.path.isdir(output_dir):
            #    os.mkdir(output_dir)

            # dowonload sentences
            head, tail = os.path.splitext(img_file_name)
            sentence_file_name = head + "txt"
            output = os.path.join(output_dir, sentence_file_name)
            if os.path.isfile(output):
                print ("Already downloaded, Skipping: %s" % output)
                continue
            print ("Downloading: %s" % output)
            with open(output,'w') as f:
                for td in tr('table tr td').items():
                    f.write(td.text() + "\n")

    def create_correspondence_data(self):
        dom = PyQuery(self.url)
        writer = csv.writer(open('correspondence.csv', 'wb'))
        for i, img in enumerate(dom('img').items()):
            img_src = img.attr['src']
            print ("%d => %s" % (i + 1, img_src))
            writer.writerow([i + 1, img_src])

if __name__=="__main__":

    # create instance
    dataset = PascalSentenceDataSet()
    # download images
    dataset.download_images()
    # download sentences
    dataset.download_sentences()
    # create correspondence data by dataset
    #dataset.create_correspondence_data()

In [1]:
!pip install tqdm

    100% |████████████████████████████████| 51kB 2.1MB/s 


In [0]:
!rm -rf titles.txt

In [68]:
#Necesitamos una funcion par crear un txt con los nombres de las imagenes
from os import listdir
from tqdm import tqdm

def extract_titles(directory, filename):
  
  files_in_directory = listdir(directory)
  #n_titles = len(files_in_directory)
  for i, name in enumerate(tqdm(files_in_directory)):
    
    data = name.split('t')[0] + '.jpg' + '\n'
    file = open(filename, 'a')
    file.write(data)
    file.close()
      
extract_titles('PascalSentenceDataSetText', 'titles.txt')


100%|██████████| 1000/1000 [00:00<00:00, 17930.12it/s]


In [70]:
!cat PascalSentenceDataSetText/2008_003607txt

A blonde dog and a black and gray dog sitting on opposite ends of a beige sofa, with a gray cat sleeping between them
A white dog, a black dog, and a gray cat lay on a white couch together near a window.
Cats and dogs getting along while sleeping on the couch.
Two dogs and a cat resting on a couch.
Two dogs and one cat sitting on couch.


In [0]:
!rm -rf descriptions.txt

In [159]:
#Vamos a crear un nuevo load_descriptions que coja la primera descripcion de cada uno de estos txt

import string
import re

def load_descriptions(directory, filename):
	  
  files_in_directory = listdir(directory)
  #n_titles = len(files_in_directory)
  #table = str.maketrans('', '', string.punctuation)
  for i, name in enumerate(tqdm(files_in_directory)):
   
    file = open(directory + '/' + name, 'r')
    data = file.read().split('\n')[0].lower()
    data = re.sub(r'[^\w\s]','',data)
    data = re.sub(r'\b\w{1,1}\b', '', data)
    file.close()
    
    file2 = open(filename, 'a')
    file2.write(name.split('t')[0] + ' '+ data + '\n')
    file2.close 
    
load_descriptions('PascalSentenceDataSetText', 'descriptions.txt')

100%|██████████| 1000/1000 [00:00<00:00, 7151.07it/s]


In [172]:
def descDict (directory):
  
  mapping = dict()
  files_in_directory = listdir(directory)
  #n_titles = len(files_in_directory)
  for i, name in enumerate(tqdm(files_in_directory)):
   
    file = open(directory + '/' + name, 'r')
    data = file.read().split('\n')[0].lower()
    data = re.sub(r'[^\w\s]','',data)
    data = re.sub(r'\b\w{1,1}\b', '', data)
    file.close()
    
    if name.split('t')[0] not in mapping:
      mapping[name.split('t')[0]] = data
  return mapping

descriptions = descDict('PascalSentenceDataSetText')
  

100%|██████████| 1000/1000 [00:00<00:00, 14304.29it/s]


In [173]:
descriptions

{'2008_003607': ' blonde dog and  black and gray dog sitting on opposite ends of  beige sofa with  gray cat sleeping between them',
 '2008_001479': ' brown dachshund beside the door',
 '2008_007317': ' black bird sits on the edge of  piece of pavement looking to the side',
 '2008_004995': ' bicycle is parked by  shop',
 '2008_008538': ' decorated room with blue carpeting  cream sofa and chair and  patio door',
 '2008_007095': 'an airplane flies against  colorful sky with the sun rising',
 '2008_005831': ' white animal running through snow covered woods',
 '2008_007415': ' chair as the central item in the picture',
 '2008_000817': ' kitchen dinette set sits on  backyard deck',
 '2008_007693': 'lambs stand behind  barbed wire fence',
 '2008_003805': ' brown race horse jockey in green and handler',
 '2008_004363': ' bicycle on display with  shade attached',
 '2008_004176': ' group of girls are posing with two',
 '2008_001920': ' closeup picture of  desk with  computer and papers on it',
 

In [143]:
descriptions['2008_003607']

' blonde dog and  black and gray dog sitting on opposite ends of  beige sofa with  gray cat sleeping between them'

Con esto quedan preparadas las descripciones en un diccionario, una sola por imagen y sin palabras de un solo caracter.




In [2]:
from os import listdir
from pickle import load, dump
from tqdm import tqdm
from numpy import array
from numpy import argmax
from pandas import DataFrame
from nltk.translate.bleu_score import corpus_bleu
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Input, Dense, Flatten, LSTM, RepeatVector, TimeDistributed, Embedding
from keras.layers.merge import concatenate
from keras.layers.pooling import GlobalMaxPooling2D

Using TensorFlow backend.


Utilizamos una VGG16 sin la capa de clasificacion (solo queremos las caracteristicas). Este modelo es el mas utilizado para la extraccion de caracteristicas en este tipo de problemas.

In [145]:
# extract features from each photo in the directory
def extract_features(directory):
  # load the model
  in_layer = Input(shape=(224, 224, 3))
  model = VGG16(include_top=False, input_tensor=in_layer)
  print(model.summary())
  # extract features from each photo
  features = dict()

  files_in_directory = listdir(directory)
  n_images = len(files_in_directory)
  for i, name in tqdm(enumerate(files_in_directory)):
    # load an image from file
    filename = directory + '/' + name
    image = load_img(filename, target_size=(224, 224))
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    # get features
    feature = model.predict(image, verbose=0)
    # get image id
    image_id = name.split('.')[0]
    # store feature
    features[image_id] = feature
    # print('{} / {} > {}'.format(i, n_images, name))
  return features

# extract features from all images
directory = 'PascalSentenceDataSetOutput'
features = extract_features(directory)
print('Extracted Features: %d' % len(features))
# save to file
dump(features, open('features.pkl', 'wb'))

58892288/58889256 [==============================] - 1s 0us/step


0it [00:00, ?it/s]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

1000it [00:34, 29.39it/s]

Extracted Features: 1000


In [176]:
features['2008_004416']

array([[[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 1.5902635 ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           1.7550225 ,  0.        ],
         ...,
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ]],

        [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           3.4605947 ,  0.        ],
         ...,
         [ 0.        ,  0.        

Ya tenemos un archivo features.pkl con las caracteristicas de las imagenes. 

In [162]:
from pickle import load

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# split a dataset into train/test elements
def train_test_split(dataset):
	# order keys so the split is consistent
	ordered = sorted(dataset)
	# return split dataset as two new sets
	return set(ordered[:400]), set(ordered[400:500])

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split(' ')
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# store
			descriptions[image_id] = 'startseq ' + ' '.join(image_desc) + ' endseq'
	return descriptions

# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features

# load dev set
filename = 'titles.txt'
dataset = load_set(filename)
print('Dataset: %d' % len(dataset))
# train-test split
train, test = train_test_split(dataset)
print('Train=%d, Test=%d' % (len(train), len(test)))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: train=%d, test=%d' % (len(train_descriptions), len(test_descriptions)))
# photo features
train_features = load_photo_features('features.pkl', train)
test_features = load_photo_features('features.pkl', test)
print('Photos: train=%d, test=%d' % (len(train_features), len(test_features)))

Dataset: 1000
Train=400, Test=100
Descriptions: train=400, test=100
Photos: train=400, test=100


In [163]:
# Ahora codificamos nuestras descripciones a números
from keras.preprocessing.text import Tokenizer

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = list(descriptions.values())
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
 
# prepare tokenizer
tokenizer = create_tokenizer(descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 1530


In [0]:
# Y creamos las secuencias:

# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, desc, image, max_length):
	Ximages, XSeq, y = list(), list(),list()
	vocab_size = len(tokenizer.word_index) + 1
	# integer encode the description
	seq = tokenizer.texts_to_sequences([desc])[0]
	# split one sequence into multiple X,y pairs
	for i in range(1, len(seq)):
		# select
		in_seq, out_seq = seq[:i], seq[i]
		# pad input sequence
		in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
		# encode output sequence
		out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
		# store
		Ximages.append(image)
		XSeq.append(in_seq)
		y.append(out_seq)
	# Ximages, XSeq, y = array(Ximages), array(XSeq), array(y)
	return [Ximages, XSeq, y]

In [0]:
# define the captioning model
def define_model(vocab_size, max_length):
	# feature extractor (encoder)
	inputs1 = Input(shape=(7, 7, 512))
	fe1 = GlobalMaxPooling2D()(inputs1)
	fe2 = Dense(128, activation='relu')(fe1)
	fe3 = RepeatVector(max_length)(fe2)
	# embedding
	inputs2 = Input(shape=(max_length,))
	emb2 = Embedding(vocab_size, 50, mask_zero=True)(inputs2)
	emb3 = LSTM(256, return_sequences=True)(emb2)
	emb4 = TimeDistributed(Dense(128, activation='relu'))(emb3)
	# merge inputs
	merged = concatenate([fe3, emb4])
	# language model (decoder)
	lm2 = LSTM(500)(merged)
	lm3 = Dense(500, activation='relu')(lm2)
	outputs = Dense(vocab_size, activation='softmax')(lm3)
	# tie it together [image, seq] [word]
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	print(model.summary())
	return model

In [0]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, features, tokenizer, max_length, n_step):
	# loop until we finish training
	while 1:
		# loop over photo identifiers in the dataset
		keys = list(descriptions.keys())
		for i in range(0, len(keys), n_step):
			Ximages, XSeq, y = list(), list(),list()
			for j in range(i, min(len(keys), i+n_step)):
				image_id = keys[j]
				# retrieve photo feature input
				image = features[image_id][0]
				# retrieve text input
				desc = descriptions[image_id]
				# generate input-output pairs
				in_img, in_seq, out_word = create_sequences(tokenizer, desc, image, max_length)
				for k in range(len(in_img)):
					Ximages.append(in_img[k])
					XSeq.append(in_seq[k])
					y.append(out_word[k])
			# yield this batch of samples to the model
			yield [[array(Ximages), array(XSeq)], array(y)]

In [0]:
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None
 
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text
 
# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	# step over the whole set
	for key, desc in descriptions.items():
		# generate description
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		# store actual and predicted
		actual.append([desc.split()])
		predicted.append(yhat.split())
	# calculate BLEU score
	bleu = corpus_bleu(actual, predicted)
	return bleu

In [0]:
# load dev set
filename = 'titles.txt'
dataset = load_set(filename)
print('Dataset: %d' % len(dataset))
# train-test split
train, test = train_test_split(dataset)
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: train=%d, test=%d' % (len(train_descriptions), len(test_descriptions)))
# photo features
train_features = load_photo_features('features.pkl', train)
test_features = load_photo_features('features.pkl', test)
print('Photos: train=%d, test=%d' % (len(train_features), len(test_features)))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max(len(s.split()) for s in list(train_descriptions.values()))
print('Description Length: %d' % max_length)
 
# define experiment
model_name = 'baseline1'
verbose = 1
n_epochs = 50
n_photos_per_update = 2
n_batches_per_epoch = int(len(train) / n_photos_per_update)
n_repeats = 3
 
# run experiment
train_results, test_results = list(), list()
for i in range(n_repeats):
	# define the model
	model = define_model(vocab_size, max_length)
	# fit model
	model.fit_generator(data_generator(train_descriptions, train_features, tokenizer, max_length, n_photos_per_update), steps_per_epoch=n_batches_per_epoch, epochs=n_epochs, verbose=verbose)
	# evaluate model on training data
	train_score = evaluate_model(model, train_descriptions, train_features, tokenizer, max_length)
	test_score = evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)
	# store
	train_results.append(train_score)
	test_results.append(test_score)
	print('>%d: train=%f test=%f' % ((i+1), train_score, test_score))
# save results to file
df = DataFrame()
df['train'] = train_results
df['test'] = test_results
print(df.describe())
df.to_csv(model_name+'.csv', index=False)

Se obtienen los siguierntes resultados (se ha borrado el resultado de la celda, que descuadraba el notebook):

         train      test
         
count  3.000000  3.000000

mean   0.014560  0.023810

std    0.019196  0.013661

min    0.003477  0.015923

25%    0.003477  0.015923

50%    0.003477  0.015923

75%    0.020101  0.027754

max    0.036726  0.039585

El resultado es de 0,015 de puntuacion en train y 0,024 en test,  resultados mas bajos que los obtenidos en el ejemplo de clase.

Vamos a ver un ejemplo de las predicciones igual que en clase:

In [170]:
# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	# step over the whole set
	for key, desc in descriptions.items():
		# generate description
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		# store actual and predicted
		actual.append([desc.split()])
		predicted.append(yhat.split())
		print('Actual:    %s' % desc)
		print('Predicted: %s' % yhat)
		if len(actual) >= 5:
			break
	# calculate BLEU score
	bleu = corpus_bleu(actual, predicted)
	return bleu

# define experiment
model_name = 'baseline1'
verbose = 2
n_epochs = 50
n_photos_per_update = 2
n_batches_per_epoch = int(len(train) / n_photos_per_update)
n_repeats = 3
 
# run experiment
train_results, test_results = list(), list()
for i in range(n_repeats):
	# define the model
	model = define_model(vocab_size, max_length)
	# fit model
	model.fit_generator(data_generator(train_descriptions, train_features, tokenizer, max_length, n_photos_per_update), steps_per_epoch=n_batches_per_epoch, epochs=n_epochs, verbose=verbose)
	# evaluate model on training data
	train_score = evaluate_model(model, train_descriptions, train_features, tokenizer, max_length)
	test_score = evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)
	# store
	train_results.append(train_score)
	test_results.append(test_score)
	print('>%d: train=%f test=%f' % ((i+1), train_score, test_score))
# save results to file
df = DataFrame()
df['train'] = train_results
df['test'] = test_results
print(df.describe())
df.to_csv(model_name+'.csv', index=False)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 7, 7, 512)    0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 22)           0                                            
__________________________________________________________________________________________________
global_max_pooling2d_4 (GlobalM (None, 512)          0           input_8[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 22, 50)       46400       input_9[0][0]                    
__________________________________________________________________________________________________
dense_13 (

 - 29s - loss: 5.4808 - acc: 0.1138
Epoch 16/50
 - 28s - loss: 5.4791 - acc: 0.1138
Epoch 17/50
 - 29s - loss: 5.4791 - acc: 0.1138
Epoch 18/50
 - 29s - loss: 5.4847 - acc: 0.1138
Epoch 19/50
 - 29s - loss: 5.4769 - acc: 0.1138
Epoch 20/50
 - 29s - loss: 5.4760 - acc: 0.1138
Epoch 21/50
 - 28s - loss: 5.4752 - acc: 0.1138
Epoch 22/50
 - 29s - loss: 5.4765 - acc: 0.1138
Epoch 23/50
 - 29s - loss: 5.4754 - acc: 0.1138
Epoch 24/50
 - 29s - loss: 5.4745 - acc: 0.1138
Epoch 25/50
 - 29s - loss: 5.4765 - acc: 0.1138
Epoch 26/50
 - 29s - loss: 5.4703 - acc: 0.1138
Epoch 27/50
 - 29s - loss: 5.4760 - acc: 0.1138
Epoch 28/50
 - 28s - loss: 5.4744 - acc: 0.1138
Epoch 29/50
 - 29s - loss: 5.4569 - acc: 0.1138
Epoch 30/50
 - 29s - loss: 5.4741 - acc: 0.1138
Epoch 31/50
 - 28s - loss: 5.4736 - acc: 0.1138
Epoch 32/50
 - 29s - loss: 5.4731 - acc: 0.1138
Epoch 33/50
 - 28s - loss: 5.4732 - acc: 0.1138
Epoch 34/50
 - 29s - loss: 5.4722 - acc: 0.1138
Epoch 35/50
 - 29s - loss: 5.4718 - acc: 0.1138
Epoc

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Actual:    startseq  couple enjoying  sunny day on the top deck of  cruise ship endseq
Predicted: startseq endseq
Actual:    startseq the lamb is looking at the camera endseq
Predicted: startseq endseq
>1: train=0.009095 test=0.036883
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 7, 7, 512)    0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 22)           0                                            
__________________________________________________________________________________________________
global_max_pooling2d_5 (GlobalM (None, 512)          0           input_10[0][0]                   
________________________________________________________________________

 - 29s - loss: 5.4730 - acc: 0.1138
Epoch 15/50
 - 29s - loss: 5.4895 - acc: 0.1138
Epoch 16/50
 - 29s - loss: 5.4846 - acc: 0.1138
Epoch 17/50
 - 29s - loss: 5.4734 - acc: 0.1138
Epoch 18/50
 - 29s - loss: 5.4850 - acc: 0.1138
Epoch 19/50
 - 29s - loss: 5.4818 - acc: 0.1138
Epoch 20/50
 - 29s - loss: 5.4807 - acc: 0.1138
Epoch 21/50
 - 29s - loss: 5.4799 - acc: 0.1138
Epoch 22/50
 - 29s - loss: 5.4790 - acc: 0.1138
Epoch 23/50
 - 29s - loss: 5.4784 - acc: 0.1138
Epoch 24/50
 - 29s - loss: 5.4778 - acc: 0.1138
Epoch 25/50
 - 28s - loss: 5.4773 - acc: 0.1138
Epoch 26/50
 - 28s - loss: 5.4768 - acc: 0.1138
Epoch 27/50
 - 29s - loss: 5.4764 - acc: 0.1138
Epoch 28/50
 - 29s - loss: 5.4759 - acc: 0.1138
Epoch 29/50
 - 29s - loss: 5.4755 - acc: 0.1138
Epoch 30/50
 - 29s - loss: 5.4751 - acc: 0.1138
Epoch 31/50
 - 29s - loss: 5.4747 - acc: 0.1138
Epoch 32/50
 - 29s - loss: 5.4743 - acc: 0.1138
Epoch 33/50
 - 29s - loss: 5.4739 - acc: 0.1138
Epoch 34/50
 - 29s - loss: 5.4736 - acc: 0.1138
Epoc

 - 29s - loss: 5.5239 - acc: 0.1138
Epoch 3/50
 - 29s - loss: 5.5977 - acc: 0.1138
Epoch 4/50
 - 29s - loss: 5.5199 - acc: 0.1138
Epoch 5/50
 - 29s - loss: 5.5134 - acc: 0.1138
Epoch 6/50
 - 29s - loss: 5.5123 - acc: 0.1138
Epoch 7/50
 - 29s - loss: 5.4857 - acc: 0.1138
Epoch 8/50
 - 29s - loss: 5.5021 - acc: 0.1138
Epoch 9/50
 - 29s - loss: 5.4981 - acc: 0.1138
Epoch 10/50
 - 29s - loss: 5.4942 - acc: 0.1138
Epoch 11/50
 - 29s - loss: 5.4917 - acc: 0.1138
Epoch 12/50
 - 29s - loss: 5.4896 - acc: 0.1138
Epoch 13/50
 - 29s - loss: 5.4880 - acc: 0.1138
Epoch 14/50
 - 29s - loss: 5.4866 - acc: 0.1138
Epoch 15/50
 - 29s - loss: 5.4854 - acc: 0.1138
Epoch 16/50
 - 29s - loss: 5.4842 - acc: 0.1138
Epoch 17/50
 - 29s - loss: 5.4832 - acc: 0.1138
Epoch 18/50
 - 29s - loss: 5.4823 - acc: 0.1138
Epoch 19/50
 - 29s - loss: 5.4813 - acc: 0.1138
Epoch 20/50
 - 29s - loss: 5.4804 - acc: 0.1138
Epoch 21/50
 - 29s - loss: 5.4795 - acc: 0.1138
Epoch 22/50
 - 29s - loss: 5.4786 - acc: 0.1138
Epoch 23/50

 - 29s - loss: 5.4025 - acc: 0.1212
Epoch 48/50
 - 29s - loss: 5.3938 - acc: 0.1228
Epoch 49/50
 - 29s - loss: 5.3849 - acc: 0.1228
Epoch 50/50
 - 29s - loss: 5.3765 - acc: 0.1228
Actual:    startseq  blonde dog and  black and gray dog sitting on opposite ends of  beige sofa with  gray cat sleeping between them endseq
Predicted: startseq black endseq
Actual:    startseq  brown dachshund beside the door endseq
Predicted: startseq black endseq
Actual:    startseq  kitchen dinette set sits on  backyard deck endseq
Predicted: startseq black endseq
Actual:    startseq  brown race horse jockey in green and handler endseq
Predicted: startseq black endseq
Actual:    startseq  group of girls are posing with two endseq
Predicted: startseq black endseq
Actual:    startseq  bicycle is parked by  shop endseq
Predicted: startseq black endseq
Actual:    startseq  bicycle on display with  shade attached endseq
Predicted: startseq black endseq
Actual:    startseq  bus at  bus stop endseq
Predicted: sta

Al final hemos conseguido algo mejor que "startseq -> endseq" y ya predice la palabra "black" en todos los casos probados. Una pequeña mejoría pero sigue siendo un intento fallido.

3.Vamos por ultimo a replicar el ejemplo de clase con flickr8k. Una vez que este funcionando trataremos de conseguir un mejor resultado. Lo que intentaremos es:

    -Ampliar la muestra de train, tal y como hemos hecho en el ejemplo anterior. 
    
    -Al margen de esto no vamos a tocar la VGG para extraer las caracteristicas [Entiendo que inicializar los pesos con transfer learning o fine tuning no ayuda ya que en cualquier caso no estamos utilizando la capa clasificadora de la CNN]. 
    
    -Centramos nuestros esfuerzos en el modelo que predice las descripciones de las fotografias. Este modelo consta de un encoder (capa densa), un procesador de secuencias (capa LSTM) y un interprete de lenguaje (capas LSTM y densa).
    
    -Actuaremos por lo tanto en los parametros del modelo. Para ello intentaremos optimizarlos implementando hyper-opt.

In [3]:
!wget http://nlp.cs.illinois.edu/HockenmaierGroup/Framing_Image_Description/Flickr8k_text.zip
!unzip -o Flickr8k_text.zip

--2018-07-01 10:20:20--  http://nlp.cs.illinois.edu/HockenmaierGroup/Framing_Image_Description/Flickr8k_text.zip
Resolving nlp.cs.illinois.edu (nlp.cs.illinois.edu)... 192.17.58.132
Connecting to nlp.cs.illinois.edu (nlp.cs.illinois.edu)|192.17.58.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2340801 (2.2M) [application/zip]
Saving to: ‘Flickr8k_text.zip’

Flickr8k_text.zip   100%[===================>]   2.23M  4.90MB/s    in 0.5s    

2018-07-01 10:20:21 (4.90 MB/s) - ‘Flickr8k_text.zip’ saved [2340801/2340801]

Archive:  Flickr8k_text.zip
  inflating: CrowdFlowerAnnotations.txt  
  inflating: ExpertAnnotations.txt   
  inflating: Flickr8k.lemma.token.txt  
   creating: __MACOSX/
  inflating: __MACOSX/._Flickr8k.lemma.token.txt  
  inflating: Flickr8k.token.txt      
  inflating: Flickr_8k.devImages.txt  
  inflating: Flickr_8k.testImages.txt  
  inflating: Flickr_8k.trainImages.txt  
  inflating: readme.txt              


In [0]:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
filename = 'Flickr8k.token.txt'
# load descriptions
doc = load_doc(filename)

In [5]:
len(doc)

3395237

In [6]:
# extract descriptions for images
def load_descriptions(doc):
	mapping = dict()
	# process lines
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		if len(line) < 2:
			continue
		# take the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# remove filename from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# store the first description for each image
		if image_id not in mapping:
			mapping[image_id] = image_desc
	return mapping
 
# parse descriptions
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

Loaded: 8092 


In [7]:
import string

def clean_descriptions(descriptions):
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for key, desc in descriptions.items():
		# tokenize
		desc = desc.split()
		# convert to lower case
		desc = [word.lower() for word in desc]
		# remove punctuation from each token
		desc = [w.translate(table) for w in desc]
		# remove hanging 's' and 'a'
		desc = [word for word in desc if len(word)>1]
		# store as string
		descriptions[key] =  ' '.join(desc)

# clean descriptions
clean_descriptions(descriptions)
# summarize vocabulary
all_tokens = ' '.join(descriptions.values()).split()
vocabulary = set(all_tokens)
print('Vocabulary Size: %d' % len(vocabulary))

Vocabulary Size: 4484


In [0]:
!rm -rf descriptions.txt

In [0]:
# save descriptions to file, one per line
def save_doc(descriptions, filename):
	lines = list()
	for key, desc in descriptions.items():
		lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

# save descriptions
save_doc(descriptions, 'descriptions.txt')

In [9]:
!wget http://nlp.cs.illinois.edu/HockenmaierGroup/Framing_Image_Description/Flickr8k_Dataset.zip
!unzip -q Flickr8k_Dataset.zip

--2018-07-01 10:20:41--  http://nlp.cs.illinois.edu/HockenmaierGroup/Framing_Image_Description/Flickr8k_Dataset.zip
Resolving nlp.cs.illinois.edu (nlp.cs.illinois.edu)... 192.17.58.132
Connecting to nlp.cs.illinois.edu (nlp.cs.illinois.edu)|192.17.58.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115419746 (1.0G) [application/zip]
Saving to: ‘Flickr8k_Dataset.zip’

Flickr8k_Dataset.zi 100%[===================>]   1.04G  82.3MB/s    in 14s     

2018-07-01 10:20:54 (78.3 MB/s) - ‘Flickr8k_Dataset.zip’ saved [1115419746/1115419746]



In [10]:
!ls -la

total 1102512
drwxr-xr-x 1 root root       4096 Jul  1 10:20 .
drwxr-xr-x 1 root root       4096 Jul  1 10:14 ..
drwx------ 4 root root       4096 Jul  1 10:17 .cache
drwxr-xr-x 3 root root       4096 Jul  1 10:17 .config
-rw-r--r-- 1 root root    2918552 Oct 14  2013 CrowdFlowerAnnotations.txt
drwxr-xr-x 3 root root       4096 Jun 28 16:55 datalab
-rw-r--r-- 1 root root     587144 Jul  1 10:20 descriptions.txt
-rw-r--r-- 1 root root     346674 Oct 14  2013 ExpertAnnotations.txt
drwxr-xr-x 2 root root     430080 Oct  3  2012 Flicker8k_Dataset
-rw-r--r-- 1 root root 1115419746 Oct 24  2013 Flickr8k_Dataset.zip
-rw-r--r-- 1 root root      25801 Oct 10  2013 Flickr_8k.devImages.txt
-rw-r--r-- 1 root root    3244761 Feb 16  2012 Flickr8k.lemma.token.txt
-rw-r--r-- 1 root root      25775 Oct 10  2013 Flickr_8k.testImages.txt
-rw-r--r-- 1 root root    2340801 Oct 28  2013 Flickr8k_text.zip
-rw-r--r-- 1 root root    3395237 Oct 14  2013 Flickr8k.token.txt
-rw-r--r-- 1 root root     154678 Oct

In [11]:
# extract features from each photo in the directory
def extract_features(directory):
  # load the model
  in_layer = Input(shape=(224, 224, 3))
  model = VGG16(include_top=False, input_tensor=in_layer)
  print(model.summary())
  # extract features from each photo
  features = dict()

  files_in_directory = listdir(directory)
  n_images = len(files_in_directory)
  for i, name in tqdm(enumerate(files_in_directory)):
    # load an image from file
    filename = directory + '/' + name
    image = load_img(filename, target_size=(224, 224))
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    # get features
    feature = model.predict(image, verbose=0)
    # get image id
    image_id = name.split('.')[0]
    # store feature
    features[image_id] = feature
    # print('{} / {} > {}'.format(i, n_images, name))
  return features

# extract features from all images
directory = 'Flicker8k_Dataset'
features = extract_features(directory)
print('Extracted Features: %d' % len(features))
# save to file
dump(features, open('features.pkl', 'wb'))

58892288/58889256 [==============================] - 1s 0us/step


0it [00:00, ?it/s]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

8091it [04:24, 30.56it/s]


Extracted Features: 8091


In [12]:
from pickle import load

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# split a dataset into train/test elements
def train_test_split(dataset):
	# order keys so the split is consistent
	ordered = sorted(dataset)
	# return split dataset as two new sets
	return set(ordered[:400]), set(ordered[400:500])

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# store
			descriptions[image_id] = 'startseq ' + ' '.join(image_desc) + ' endseq'
	return descriptions

# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features

# load dev set
filename = 'Flickr_8k.devImages.txt'
dataset = load_set(filename)
print('Dataset: %d' % len(dataset))
# train-test split
train, test = train_test_split(dataset)
print('Train=%d, Test=%d' % (len(train), len(test)))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: train=%d, test=%d' % (len(train_descriptions), len(test_descriptions)))
# photo features
train_features = load_photo_features('features.pkl', train)
test_features = load_photo_features('features.pkl', test)
print('Photos: train=%d, test=%d' % (len(train_features), len(test_features)))

Dataset: 1000
Train=400, Test=100
Descriptions: train=400, test=100
Photos: train=400, test=100


In [13]:
# Ahora codificamos nuestras descripciones a números
from keras.preprocessing.text import Tokenizer

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = list(descriptions.values())
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
 
# prepare tokenizer
tokenizer = create_tokenizer(descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 4485


In [0]:
# Y creamos las secuencias:

# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, desc, image, max_length):
	Ximages, XSeq, y = list(), list(),list()
	vocab_size = len(tokenizer.word_index) + 1
	# integer encode the description
	seq = tokenizer.texts_to_sequences([desc])[0]
	# split one sequence into multiple X,y pairs
	for i in range(1, len(seq)):
		# select
		in_seq, out_seq = seq[:i], seq[i]
		# pad input sequence
		in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
		# encode output sequence
		out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
		# store
		Ximages.append(image)
		XSeq.append(in_seq)
		y.append(out_seq)
	# Ximages, XSeq, y = array(Ximages), array(XSeq), array(y)
	return [Ximages, XSeq, y]

In [0]:
# define the captioning model
def define_model(vocab_size, max_length):
	# feature extractor (encoder)
	inputs1 = Input(shape=(7, 7, 512))
	fe1 = GlobalMaxPooling2D()(inputs1)
	fe2 = Dense(128, activation='relu')(fe1)
	fe3 = RepeatVector(max_length)(fe2)
	# embedding
	inputs2 = Input(shape=(max_length,))
	emb2 = Embedding(vocab_size, 50, mask_zero=True)(inputs2)
	emb3 = LSTM(256, return_sequences=True)(emb2)
	emb4 = TimeDistributed(Dense(128, activation='relu'))(emb3)
	# merge inputs
	merged = concatenate([fe3, emb4])
	# language model (decoder)
	lm2 = LSTM(500)(merged)
	lm3 = Dense(500, activation='relu')(lm2)
	outputs = Dense(vocab_size, activation='softmax')(lm3)
	# tie it together [image, seq] [word]
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	print(model.summary())
	return model

In [0]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, features, tokenizer, max_length, n_step):
	# loop until we finish training
	while 1:
		# loop over photo identifiers in the dataset
		keys = list(descriptions.keys())
		for i in range(0, len(keys), n_step):
			Ximages, XSeq, y = list(), list(),list()
			for j in range(i, min(len(keys), i+n_step)):
				image_id = keys[j]
				# retrieve photo feature input
				image = features[image_id][0]
				# retrieve text input
				desc = descriptions[image_id]
				# generate input-output pairs
				in_img, in_seq, out_word = create_sequences(tokenizer, desc, image, max_length)
				for k in range(len(in_img)):
					Ximages.append(in_img[k])
					XSeq.append(in_seq[k])
					y.append(out_word[k])
			# yield this batch of samples to the model
			yield [[array(Ximages), array(XSeq)], array(y)]

In [0]:
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None
 
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text
 
# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	# step over the whole set
	for key, desc in descriptions.items():
		# generate description
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		# store actual and predicted
		actual.append([desc.split()])
		predicted.append(yhat.split())
	# calculate BLEU score
	bleu = corpus_bleu(actual, predicted)
	return bleu

In [0]:
# load dev set
filename = 'Flickr_8k.devImages.txt'
dataset = load_set(filename)
print('Dataset: %d' % len(dataset))
# train-test split
train, test = train_test_split(dataset)
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: train=%d, test=%d' % (len(train_descriptions), len(test_descriptions)))
# photo features
train_features = load_photo_features('features.pkl', train)
test_features = load_photo_features('features.pkl', test)
print('Photos: train=%d, test=%d' % (len(train_features), len(test_features)))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max(len(s.split()) for s in list(train_descriptions.values()))
print('Description Length: %d' % max_length)
 
# define experiment
model_name = 'baseline1'
verbose = 1
n_epochs = 50
n_photos_per_update = 2
n_batches_per_epoch = int(len(train) / n_photos_per_update)
n_repeats = 3
 
# run experiment
train_results, test_results = list(), list()
for i in range(n_repeats):
	# define the model
	model = define_model(vocab_size, max_length)
	# fit model
	model.fit_generator(data_generator(train_descriptions, train_features, tokenizer, max_length, n_photos_per_update), steps_per_epoch=n_batches_per_epoch, epochs=n_epochs, verbose=verbose)
	# evaluate model on training data
	train_score = evaluate_model(model, train_descriptions, train_features, tokenizer, max_length)
	test_score = evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)
	# store
	train_results.append(train_score)
	test_results.append(test_score)
	print('>%d: train=%f test=%f' % ((i+1), train_score, test_score))
# save results to file
df = DataFrame()
df['train'] = train_results
df['test'] = test_results
print(df.describe())
df.to_csv(model_name+'.csv', index=False)

Conseguimos unas puntuaciones de 0,015 y 0,020, inferiores a las que se obtuvieron en clase.

          train      test
          
count  3.000000  3.000000

mean   0.015949  0.020083

std    0.010898  0.015056

min    0.009658  0.011390

25%    0.009658  0.011390

50%    0.009658  0.011390

75%    0.019095  0.024429

max    0.028533  0.037468


In [22]:
# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	# step over the whole set
	for key, desc in descriptions.items():
		# generate description
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		# store actual and predicted
		actual.append([desc.split()])
		predicted.append(yhat.split())
		print('Actual:    %s' % desc)
		print('Predicted: %s' % yhat)
		if len(actual) >= 5:
			break
	# calculate BLEU score
	bleu = corpus_bleu(actual, predicted)
	return bleu

# define experiment
model_name = 'baseline1'
verbose = 2
n_epochs = 50
n_photos_per_update = 2
n_batches_per_epoch = int(len(train) / n_photos_per_update)
n_repeats = 3
 
# run experiment
train_results, test_results = list(), list()
for i in range(n_repeats):
	# define the model
	model = define_model(vocab_size, max_length)
	# fit model
	model.fit_generator(data_generator(train_descriptions, train_features, tokenizer, max_length, n_photos_per_update), steps_per_epoch=n_batches_per_epoch, epochs=n_epochs, verbose=verbose)
	# evaluate model on training data
	train_score = evaluate_model(model, train_descriptions, train_features, tokenizer, max_length)
	test_score = evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)
	# store
	train_results.append(train_score)
	test_results.append(test_score)
	print('>%d: train=%f test=%f' % ((i+1), train_score, test_score))
# save results to file
df = DataFrame()
df['train'] = train_results
df['test'] = test_results
print(df.describe())
pdf.to_csv(model_name+'.csv', index=False)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 7, 7, 512)    0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 25)           0                                            
__________________________________________________________________________________________________
global_max_pooling2d_5 (GlobalM (None, 512)          0           input_10[0][0]                   
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 25, 50)       45050       input_11[0][0]                   
__________________________________________________________________________________________________
dense_17 (

 - 33s - loss: 5.2352 - acc: 0.1048
Epoch 16/50
 - 33s - loss: 5.2584 - acc: 0.1038
Epoch 17/50
 - 33s - loss: 5.2400 - acc: 0.1039
Epoch 18/50
 - 33s - loss: 5.2511 - acc: 0.1026
Epoch 19/50
 - 33s - loss: 5.2342 - acc: 0.1032
Epoch 20/50
 - 33s - loss: 5.2363 - acc: 0.1036
Epoch 21/50
 - 33s - loss: 5.2345 - acc: 0.1016
Epoch 22/50
 - 33s - loss: 5.2307 - acc: 0.1052
Epoch 23/50
 - 33s - loss: 5.2241 - acc: 0.1054
Epoch 24/50
 - 33s - loss: 5.2199 - acc: 0.1036
Epoch 25/50
 - 33s - loss: 5.2072 - acc: 0.1039
Epoch 26/50
 - 33s - loss: 5.1992 - acc: 0.1036
Epoch 27/50
 - 33s - loss: 5.2026 - acc: 0.1062
Epoch 28/50
 - 33s - loss: 5.1883 - acc: 0.1046
Epoch 29/50
 - 33s - loss: 5.1809 - acc: 0.1033
Epoch 30/50
 - 33s - loss: 5.1828 - acc: 0.1037
Epoch 31/50
 - 33s - loss: 5.1710 - acc: 0.1049
Epoch 32/50
 - 33s - loss: 5.1696 - acc: 0.1033
Epoch 33/50
 - 33s - loss: 5.1808 - acc: 0.1030
Epoch 34/50
 - 33s - loss: 5.1715 - acc: 0.1061
Epoch 35/50
 - 33s - loss: 5.1600 - acc: 0.1062
Epoc

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Actual:    startseq child leaping toward red bed endseq
Predicted: startseq endseq
Actual:    startseq airplane controller is outside white airplane endseq
Predicted: startseq endseq
Actual:    startseq four kayakers and canoers near rock wall endseq
Predicted: startseq endseq
Actual:    startseq biker jumping extremely high endseq
Predicted: startseq endseq
>1: train=0.060907 test=0.049787
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 7, 7, 512)    0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           (None, 25)           0                                            
__________________________________________________________________________________________________
global_max_p

Epoch 14/50
 - 33s - loss: 5.2336 - acc: 0.1042
Epoch 15/50
 - 33s - loss: 5.2220 - acc: 0.1030
Epoch 16/50
 - 34s - loss: 5.2128 - acc: 0.1025
Epoch 17/50
 - 33s - loss: 5.2045 - acc: 0.1033
Epoch 18/50
 - 33s - loss: 5.2208 - acc: 0.1037
Epoch 19/50
 - 33s - loss: 5.1983 - acc: 0.1031
Epoch 20/50
 - 33s - loss: 5.2039 - acc: 0.1027
Epoch 21/50
 - 33s - loss: 5.2143 - acc: 0.1026
Epoch 22/50
 - 33s - loss: 5.2033 - acc: 0.1047
Epoch 23/50
 - 33s - loss: 5.1879 - acc: 0.1034
Epoch 24/50
 - 33s - loss: 5.2054 - acc: 0.1036
Epoch 25/50
 - 33s - loss: 5.1784 - acc: 0.1031
Epoch 26/50
 - 33s - loss: 5.1701 - acc: 0.1016
Epoch 27/50
 - 33s - loss: 5.2029 - acc: 0.1040
Epoch 28/50
 - 33s - loss: 5.1740 - acc: 0.1036
Epoch 29/50
 - 33s - loss: 5.1906 - acc: 0.1031
Epoch 30/50
 - 33s - loss: 5.1879 - acc: 0.1044
Epoch 31/50
 - 33s - loss: 5.1776 - acc: 0.1054
Epoch 32/50
 - 33s - loss: 5.1838 - acc: 0.1033
Epoch 33/50
 - 33s - loss: 5.1701 - acc: 0.1040
Epoch 34/50
 - 33s - loss: 5.1738 - acc:

 - 33s - loss: 5.4622 - acc: 0.1031
Epoch 3/50
 - 33s - loss: 5.4675 - acc: 0.1031
Epoch 4/50
 - 33s - loss: 5.4742 - acc: 0.1031
Epoch 5/50
 - 33s - loss: 5.4648 - acc: 0.1031
Epoch 6/50
 - 33s - loss: 5.4373 - acc: 0.1031
Epoch 7/50
 - 33s - loss: 5.4262 - acc: 0.1031
Epoch 8/50
 - 33s - loss: 5.4202 - acc: 0.1031
Epoch 9/50
 - 33s - loss: 5.4164 - acc: 0.1031
Epoch 10/50
 - 33s - loss: 5.4165 - acc: 0.1031
Epoch 11/50
 - 33s - loss: 5.4137 - acc: 0.1031
Epoch 12/50
 - 33s - loss: 5.4126 - acc: 0.1031
Epoch 13/50
 - 33s - loss: 5.4120 - acc: 0.1031
Epoch 14/50
 - 33s - loss: 5.4081 - acc: 0.1031
Epoch 15/50
 - 33s - loss: 5.4098 - acc: 0.1031
Epoch 16/50
 - 33s - loss: 5.4087 - acc: 0.1031
Epoch 17/50
 - 33s - loss: 5.4079 - acc: 0.1031
Epoch 18/50
 - 33s - loss: 5.4072 - acc: 0.1031
Epoch 19/50
 - 33s - loss: 5.4065 - acc: 0.1031
Epoch 20/50
 - 33s - loss: 5.4059 - acc: 0.1031
Epoch 21/50
 - 33s - loss: 5.4053 - acc: 0.1031
Epoch 22/50
 - 33s - loss: 5.4047 - acc: 0.1031
Epoch 23/50

 - 33s - loss: 5.2518 - acc: 0.1161
Epoch 48/50
 - 33s - loss: 5.2438 - acc: 0.1161
Epoch 49/50
 - 33s - loss: 5.2367 - acc: 0.1161
Epoch 50/50
 - 33s - loss: 5.2303 - acc: 0.1161
Actual:    startseq child and woman are at waters edge in big city endseq
Predicted: startseq boy endseq
Actual:    startseq boy with stick kneeling in front of goalie net endseq
Predicted: startseq boy endseq
Actual:    startseq woman crouches near three dogs in field endseq
Predicted: startseq boy endseq
Actual:    startseq boy bites hard into treat while he sits outside endseq
Predicted: startseq boy endseq
Actual:    startseq person eats takeout while watching small television endseq
Predicted: startseq boy endseq
Actual:    startseq two dogs are peeing at red fire hydrant endseq
Predicted: startseq boy endseq


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Actual:    startseq child leaping toward red bed endseq
Predicted: startseq boy endseq
Actual:    startseq airplane controller is outside white airplane endseq
Predicted: startseq boy endseq
Actual:    startseq four kayakers and canoers near rock wall endseq
Predicted: startseq boy endseq
Actual:    startseq biker jumping extremely high endseq
Predicted: startseq boy endseq
>3: train=0.053675 test=0.170668
          train      test
count  3.000000  3.000000
mean   0.043193  0.090081
std    0.024686  0.069791
min    0.014996  0.049787
25%    0.034335  0.049787
50%    0.053675  0.049787
75%    0.057291  0.110228
max    0.060907  0.170668


NameError: ignored

Con esto queda replicado el ejemplo de clase, aunque los resultados son mucho peores. Tan sólo en una de las iteraciones hemos conseguido una predicción que no sea directamente "startseq -> endseq", que en cualquier caso asignaba "boy" a todas las fotos. 

Con esto parece que la precision obtenida es muy similar a la que conseguimos previamente para el dataset de Pascal. 

Tambien llama la atencion que replicandio el ejemplo de clase se obtengan resultados mucho peores, entiendo que puede ser debido a la aleatoriedad de la iniciacion de los pesos de las redes.

Una vez llegados a este punto vamos aintentar mejorar el resultado obtenido en este ultimo caso. Para ello vamos a intentar implementar un hyper-opt sencillito para obtener los valores optimos de algun parametro de la red LSTM y conseguir unas mejores predicciones.

In [16]:
from pickle import load

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# split a dataset into train/test elements
def train_test_split(dataset):
	# order keys so the split is consistent
	ordered = sorted(dataset)
	# return split dataset as two new sets
	return set(ordered[:400]), set(ordered[400:500])

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# store
			descriptions[image_id] = 'startseq ' + ' '.join(image_desc) + ' endseq'
	return descriptions

# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features

# load dev set
filename = 'Flickr_8k.devImages.txt'
dataset = load_set(filename)
print('Dataset: %d' % len(dataset))
# train-test split
train, test = train_test_split(dataset)
print('Train=%d, Test=%d' % (len(train), len(test)))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: train=%d, test=%d' % (len(train_descriptions), len(test_descriptions)))
# photo features
train_features = load_photo_features('features.pkl', train)
test_features = load_photo_features('features.pkl', test)
print('Photos: train=%d, test=%d' % (len(train_features), len(test_features)))

Dataset: 1000
Train=400, Test=100
Descriptions: train=400, test=100
Photos: train=400, test=100


In [17]:
# Ahora codificamos nuestras descripciones a números
from keras.preprocessing.text import Tokenizer

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = list(descriptions.values())
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
 
# prepare tokenizer
tokenizer = create_tokenizer(descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 4485


In [0]:
# Y creamos las secuencias:

# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, desc, image, max_length):
	Ximages, XSeq, y = list(), list(),list()
	vocab_size = len(tokenizer.word_index) + 1
	# integer encode the description
	seq = tokenizer.texts_to_sequences([desc])[0]
	# split one sequence into multiple X,y pairs
	for i in range(1, len(seq)):
		# select
		in_seq, out_seq = seq[:i], seq[i]
		# pad input sequence
		in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
		# encode output sequence
		out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
		# store
		Ximages.append(image)
		XSeq.append(in_seq)
		y.append(out_seq)
	# Ximages, XSeq, y = array(Ximages), array(XSeq), array(y)
	return [Ximages, XSeq, y]

In [20]:
!pip install networkx==1.11 # para instala hyperopt correctamente, si no, da errores
!pip install hyperopt

    100% |████████████████████████████████| 1.3MB 6.6MB/s 
  Found existing installation: networkx 2.1
    Uninstalling networkx-2.1:
      Successfully uninstalled networkx-2.1
    100% |████████████████████████████████| 102kB 2.5MB/s 
    100% |████████████████████████████████| 163kB 4.9MB/s 
    100% |████████████████████████████████| 409kB 5.4MB/s 
  Running setup.py bdist_wheel for hyperopt ... - \ | done
  Stored in directory: /content/.cache/pip/wheels/32/69/f5/3267146c22e76dbf8c5e13a535d3c00b9efabe58883a0da65d
Successfully built hyperopt


In [21]:
# define the captioning model
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, rand
from keras import optimizers

space = {
    'optimizer': hp.choice('optimizer', ['adadelta','adam','rmsprop']),
    'neuronas_dense': hp.choice('neuronas_dense', [128, 512]),
    'units_LSTM': hp.choice('units_LSTM', [256, 500]),
}

def prueba(a):
  #feature extractor (encoder)
  inputs1 = Input(shape=(7, 7, 512))
  fe1 = GlobalMaxPooling2D()(inputs1)
  #fe2 = Dense(128, activation='relu')(fe1)
  fe2 = Dense(a['neuronas_dense'], activation='relu')(fe1)
  fe3 = RepeatVector(max_length)(fe2)
  # embedding
  inputs2 = Input(shape=(max_length,))
  emb2 = Embedding(vocab_size, 50, mask_zero=True)(inputs2)
  #emb3 = LSTM(256, return_sequences=True)(emb2)
  emb3 = LSTM(a['units_LSTM'], return_sequences=True)(emb2)
  emb4 = TimeDistributed(Dense(128, activation='relu'))(emb3)
  # merge inputs
  merged = concatenate([fe3, emb4])
  # language model (decoder)
  #lm2 = LSTM(500)(merged)
  #lm3 = Dense(500, activation='relu')(lm2)
  lm2 = LSTM(a['units_LSTM'])(merged)
  lm3 = Dense(a['neuronas_dense'], activation='relu')(lm2)
  outputs = Dense(vocab_size, activation='softmax')(lm3)
  # tie it together [image, seq] [word]
  model = Model(inputs=[inputs1, inputs2], outputs=outputs)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  print(model.summary())

trials = Trials()
best = fmin(prueba, space, algo=tpe.suggest, max_evals=10, trials=trials)
print(best)





__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 7, 7, 512)    0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 25)           0                                            
__________________________________________________________________________________________________
global_max_pooling2d_4 (GlobalM (None, 512)          0           input_8[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 25, 50)       45050       input_9[0][0]                    
__________________________________________________________________________________________________
dense_13 (

TypeError: ignored

Lamentablemente no he sido capaza de hacer funcionar las iteraciones para encontrar el optimo de los parametros. Una serie de pruebas y mirar la documentacion no me ha sido suficiente para conseguirlo y con el tiempo tan limitado del que disponia tengo que dejarlo aqui.

Dado que los resultados del programa no han sido buenos paso al menos a explicar en detalle la practica:

El primer paso de la practica es seleccionar un dataset. Para ello he probado con 3 opciones.

En primer lugar, flickr30k. Se trata de un dataset de imagenes de flickr con 5 descripciones cada una. Este dataset me parecia interesante ya que era diferente que el del ejemplo y requeria algo de trabajo extra, como crear un archivo con los titulos de las imagenes. Tuve que abandonarlo ya que tenia demasiadas imagenes y no fui capaz de extraer sus caracteristicas en google colab.

En segundo lugar, visto ese problema, me decidi por un dataset distinto al del ejemplo y que tuviera menos imagenes: Pascal Sentence. Este dataset requeria bastante trabajo adicional dado que no cuenta con enlaces de descarga como los de flickr. Este problema lo aborde con un codigo de web scrapping para descargar por un lado las imagenes y por otro los textos. Los textos los almacene como clave-valor quedandome con solo la primera descripcion de cada imagen, como habiamos hecho en el ejemplo en clase. La preparacion de los datos fue bien y pude correr el modelo de clase hasta el final con minimos ajustes pero los resultados no fueron buenos asi que decidi como ultimo paso replicar el ejemplo de clase y tratar de mejorarlo.

En ultimo lugar, por lo tanto, utilice flickr8k al igual que en clase. Ya vimos en el ejemplo como preparara los datos de modo que esta parte no supuso ningun problema. Tras este paso obtengo un total de 8092 fotos con una descripcion cada una, asociada a una clave que es el identificador de la foto y un vocabulario de 4484 palabras diferentes.

El siguiente paso de la practica es obtener las caracteristicas de las fotos con la funcion extract_features. La idea de este paso es "separar" esta primera parte del procesado de las fotos del modelo, guardando los resultados en un archivo, para que el propio modelo pueda computar mas rapido al no tener que examimar las fotos y poder cargar sus caracteristicas directamente desde el archivo.



Para poder extraer las caracteristicas de las fotos, en vez de entrenar una red con miles de imagenes para que aprenda, utilizamos una de las arquitecturas ya entrenadas que vimos en clase el quinto dia. En el ejemplo usamos la VGG16 y por lo que he visto en algunos de los otros ejemplos de internet para problemas de este tipo parece que es la arquitectura mas comun en este caso.

Una cosa importante es que al utilizar la arquitectura de la VGG16 lo hacemos sin las capas finales, que son las que toman el trabajo del resto de la red (en esencia, la extraccion de caracteristicas) y lo utilizan para clasificar la fotografia. En este caso la clasificacion no nos interesa y son precisamente las caracteristicas de cada una de las fotos lo que queremos, para poder introducirlas en el modelo que vamos a crear para describir las imagenes.

Con esta funcion extract_features conseguimos un archivo features.pkl con un array opor cada una de las imagenes que contiene las caracteristicas que la red convolucional ha obtenido de ellas.

El siguiente paso en el ejemplo es elegir las muestras del dataset con las que vamos a trabajar. Para ello se carga un subconjunto del dataset de 1000 imagenes y de entre estas se seleccionan las 400 primeras como conjunto de train y las 100 siguientes como conjunto de test.

A continuacion hay que preparar los conjuntos de train y test. Hemos elegido una serie de fotografias (400 y 100), lo siguiente es cargar del archivo de descripciones el texto que acompaña a cada una de esas fotos. Este texto es lo que el modelo va a tener que predecir por lo que tenemos que introducirlo de forma que la red pueda aprenderlo y reproducirlo.

Para ello en el ejemplo de clase se introduce la descripcion como una secuencia de tokens o palabras, que siempre comienza con el token "startseq" y termina con "endseq". Ademas estas secuencias de palabras tienen que convertirse en enteros para que el modelo las entienda. Para ello se utiliza la funcion tokenizer que incorpora keras y que codifica cada palabra del vocabulario del que disponiamos presente en las descricpiones.

La siguiente funcion importante del ejemplo es create_sequences. Esta funcion toma como parametros el tokenizer, la descripcion y las caracteristicas de una foto. Esta funcion sirve para entrenar el modelo, dando una palabra de la secuencia como output dada la entrada.

Con esto solo queda definir el modelo.

El model consta de 3 partes:

Una primera de extraccion de caracteristicas y que ya separamos del modelo, utilizando una arquitectura VGG16.

Una segunda que procesa la secuencia de texto mediante una red recurrente (capas: embedding>LSTM>dense).

Una final que actua como interprete, recogiendo los resultados de las capas anteriores para dar una prediccion de la descripcion de la imagen. (capas: LSTM>dense).

El modelo se compila con una funcion de activacion softmax y minimizando una funcion de perdidas categorical cross entropy, dado que las predicciones tienen que hacerse de entre las palabras definidas en el vocabulario. 



El ultimo paso es evaluar el modelo. Para ello lo entrenamos con la muestra de train y se compara la descripcion real con la predecida, obteniendo una puntuacion segun la clasificacion BLEU (un indicador de 0 a 1 que representa lo parecidos que son dos textos).

Para generar la descripcion predecida se llama al modelo recursivamente, de modo que genere la palabra siguiente de la secuencia hasta obtener endseq.

Eso es todo. Me sabe mal no haber conseguido una prediccion decente pero he invertido todo el tiempo que me ha sido posible esta semana y hasta aqui he podido llegar. 

En cualquier caso han sido unas sesiones muy interesantes y me quedo alucinado de lo que es capaz de hacer keras con unas pocas lineas de codigo y una arquitectura pre entrenada (y una gpu!).